#### Importing data files from lending club data
##### Source: https://lendingclub.com/info/download-data.action


In [1]:
from pyspark.sql import functions as fn, Row
import pandas as pd
import numpy as np

#d1 = pd.read_csv('LoanStats_2017Q1.csv', low_memory = False)
#d2 = pd.read_csv('LoanStats_2017Q2.csv', low_memory = False)
#d3 = pd.read_csv('LoanStats_2017Q3.csv', low_memory = False)
#d4 = pd.read_csv('LoanStats_2017Q4.csv', low_memory = False)
#d5 = pd.read_csv('LoanStats_2018Q1.csv', low_memory = False)
#d6 = pd.read_csv('LoanStats_2018Q2.csv', low_memory = False)

#### Combining Data to create a master file

In [2]:
#masterFile = pd.concat([d1,d2,d3,d4,d5,d6], axis=0)

In [3]:
#masterFile.to_csv("data_new.csv")

In [4]:
#To release memory
#del d1,d2,d3,d4,d5,d6,masterFile

In [5]:
df = pd.read_csv('data_new.csv', low_memory = False)
df.shape

(682215, 145)

In [6]:
df_backup = df
#df = df_backup

#### Exploring null values in the dataset and excluding columns with all nulls

In [7]:
null_value_list = df.apply(lambda x: sum(x.isnull()))
null_value_list = null_value_list[null_value_list!= len(df)]
df = df[null_value_list.index]

#### Scope constraint : Consider only individual loan application type

In [8]:
df = df[df['application_type']=='Individual']
df=df.drop('application_type',axis=1)
df.shape

(605372, 140)

#### Feature Engineering

In [9]:
df['emp_length'] = df.emp_length.str.replace('+','')
df['emp_length'] = df.emp_length.str.replace('<','')
df['emp_length'] = df.emp_length.str.replace('years','')
df['emp_length'] = df.emp_length.str.replace('year','')
df['emp_length'].unique()

array(['10 ', '3 ', '9 ', '4 ', '6 ', '8 ', '1 ', ' 1 ', nan, '5 ', '2 ',
       '7 '], dtype=object)

#### Feature Engineering Contd.

In [10]:
df = df.dropna(subset=['emp_length']) #scope constraint : Remove null emp_length
df['emp_length'] = df['emp_length'].astype(int)
df.shape
df['emp_length'].unique()

array([10,  3,  9,  4,  6,  8,  1,  5,  2,  7], dtype=int64)

In [11]:
df.shape

(564420, 140)

#### Exclude nas from avg_cur_bal feature

In [12]:
df = df.dropna(subset=['avg_cur_bal']) #scope constraint : Remove null avg_cur_bal
df.shape

(564400, 140)

#### Replace % in int_rate and convert type to float

In [13]:
df['int_rate'] = df.int_rate.str.replace('%','').astype(float)

#### Replacing loan_status types with 'Default' in order to have only 2 types ot Loan Statuses :
* Default
* Fully Paid

In [14]:
df['loan_status'] = df['loan_status'].replace({'Current':'Default',  'In Grace Period':'Default',
                           'Late (31-120 days)':'Default','Late (16-30 days)':'Default','Charged Off':'Default'})
df['loan_status'].unique()

array(['Default', 'Fully Paid'], dtype=object)

#### Converting loan term feature to int

In [15]:
df['term'] = df.term.str.replace('months','').astype(int)
df['term'].unique()

array([36, 60], dtype=int64)

In [16]:
df.shape

(564400, 140)

#### Creating Master File to perform feature selection

In [17]:
masterFile = df
masterFile.to_csv("data_master.csv")